# Sample Queries

In [7]:
__author__ = "Christine Mendoza, \
    with some sqlite3-related code from Dr. Gary Bishop's / Dr. John Majikes' UNC Chapel Hill COMP421 (Databases) class"

Before going further, make sure to define your file paths.

In [8]:
DB_PATH: str = "./analysis.db"

Set up your database connection.

In [9]:
import sqlite3
db = sqlite3.Connection(DB_PATH)
cursor = db.cursor()

Create one last table for marking papers which have been completed.

In [10]:
LIST_TABLES = """
    SELECT name
        FROM sqlite_master
        WHERE type='table' AND
                name NOT LIKE 'sqlite_%'
"""
LIST_SCHEMAS = """
    SELECT sql
        FROM sqlite_master
        WHERE type='table' AND
                name NOT LIKE 'sqlite_%'
"""
NODES = """
    SELECT node_id, word FROM Nodes;
"""
QUERY_LIST_ALL_EDGES = """
    WITH Source_Text_Info AS (SELECT node_id, word FROM Nodes),
        Target_Text_Info AS (SELECT node_id, word FROM Nodes)
    SELECT E.edge_id, E.source_node, S.word, E.target_node, T.word, E.edge_type 
        FROM Edges E, Source_Text_Info S, Target_Text_Info T
        WHERE E.source_node = S.node_id
            AND E.target_node = T.node_id
        ORDER BY source_node;
"""
QUERY_LIST_ALL_EDGES_NO_ID = """
    WITH Source_Text_Info AS (SELECT node_id, word FROM Nodes),
        Target_Text_Info AS (SELECT node_id, word FROM Nodes)
    SELECT S.word, T.word 
        FROM Edges E, Source_Text_Info S, Target_Text_Info T
        WHERE E.source_node = S.node_id
            AND E.target_node = T.node_id
        ORDER BY source_node;
"""
EDGES = """
    WITH Source_Text_Info AS (SELECT node_id, word FROM Nodes),
        Target_Text_Info AS (SELECT node_id, word FROM Nodes)
    SELECT S.word, T.word, Sc.Title
        FROM Edges E, Source_Text_Info S, Target_Text_Info T, Literature_Sources_Edges L, Scopus_Info Sc
        WHERE E.source_node = S.node_id
            AND E.target_node = T.node_id
            AND E.edge_id = L.edge_id
            AND L.paper_id = Sc.paper_id
        ORDER BY E.source_node;
"""
ARTICLES = """
    SELECT No.word, Sc.Title
        FROM Scopus_Info Sc, Literature_Sources_Nodes L, Nodes No
        WHERE Sc.paper_id = L.paper_id
            AND L.node_id = No.node_id
        ORDER BY No.node_id;
"""
NUM_SOURCES_NODES = """
    SELECT No.node_id, No.word, COUNT(L.paper_id) AS source_count
        FROM Literature_Sources_Nodes L, Nodes No
        WHERE L.node_id = No.node_id
        GROUP BY No.node_id
        ORDER BY source_count DESC, No.word;
"""
NUM_LITERATURE_SOURCES_NODES = """
    SELECT COUNT(*) FROM Literature_Sources_Nodes;
"""
NUM_LITERATURE_SOURCES_EDGES = """
    SELECT COUNT(*) FROM Literature_Sources_Edges;
"""
NUM_NODES = """
    SELECT COUNT(*) FROM Nodes;
"""
NUM_EDGES = """
    SELECT COUNT(*) FROM Edges;
"""
TABLES = """
    SELECT *
        FROM sqlite_master
        WHERE type='table' AND
            name NOT LIKE 'sqlite_%'
"""
FOCUS_POPULATION = """
    SELECT Re.Focus_Population, No.word
        FROM Review_Notes Re, Nodes No, Literature_Sources_Nodes Li
        WHERE Re.paper_id = Li.paper_id
            AND No.node_id = Li.node_id
            AND Re.Focus_Population like '%visual%';
"""
COUNTRY = """
    SELECT Re.Study_Country_ies, No.word
        FROM Review_Notes Re, Nodes No, Literature_Sources_Nodes Li
        WHERE Re.paper_id = Li.paper_id
            AND No.node_id = Li.node_id
            AND Re.Study_Country_ies like '%India%';
"""

QUERY_UPDATE_REVIEW_STATUS = """
    UPDATE Completed
    SET completed = ?
    WHERE paper_id = ?;
"""

SPECIFIC_ARTICLES = """
    SELECT S.paper_id
        FROM Review_Notes S WHERE 
        S.Focus_Location like'%education%' OR 
        S.Focus_Location like'%school%' OR 
        S.Focus_Location like'%university%' OR 
        S.Focus_Location like'%Higher Ed%';
"""

SPECIFIC_ARTICLES_NUM_SOURCES = """
    WITH Specific_Articles AS (SELECT Re.paper_id 
                                FROM Review_Notes Re WHERE 
                                Re.Focus_Location not like'%indoor %' AND 
                                Re.Focus_Location not like'%building%')
    SELECT No.node_id, No.word, COUNT(L.paper_id) AS source_count
        FROM Literature_Sources_Nodes L, Nodes No, Specific_Articles Sp
        WHERE L.node_id = No.node_id
            AND L.paper_id = Sp.paper_id
        GROUP BY No.word
        ORDER BY source_count DESC, No.word;
"""
SPECIFIC_ARTICLES_NUM_SOURCES_NO_ID = """
    WITH Specific_Articles AS (SELECT Re.paper_id 
                                FROM Review_Notes Re WHERE 
                                Re.Focus_Location like'%indoor %' OR 
                                Re.Focus_Location like'%building%')
    SELECT No.word, COUNT(L.paper_id) AS source_count
        FROM Literature_Sources_Nodes L, Nodes No, Specific_Articles Sp
        WHERE L.node_id = No.node_id
            AND L.paper_id = Sp.paper_id
        GROUP BY No.word
        ORDER BY source_count DESC, No.word;
"""
SPECIFIC_ARTICLES_NUM_LITERATURE_SOURCES_NODES = """
    WITH Specific_Articles AS (SELECT Re.paper_id 
                                FROM Review_Notes Re WHERE 
                                Re.Focus_Location not like'%indoor %' AND 
                                Re.Focus_Location not like'%building%')
    SELECT COUNT(*) FROM Literature_Sources_Nodes L, Specific_Articles Sp
        WHERE L.paper_id = Sp.paper_id;
"""
SPECIFIC_ARTICLES_NUM_LITERATURE_SOURCES_EDGES = """
    WITH Specific_Articles AS (SELECT Re.paper_id 
                                FROM Review_Notes Re WHERE 
                                Re.Focus_Location not like'%indoor %' AND 
                                Re.Focus_Location not like'%building%')
    SELECT COUNT(*) FROM Literature_Sources_Edges L, Specific_Articles Sp
        WHERE L.paper_id = Sp.paper_id;
"""

SPECIFIC_ARTICLES_EDGES = """
    WITH Source_Text_Info AS (SELECT node_id, word FROM Nodes),
        Target_Text_Info AS (SELECT node_id, word FROM Nodes),
        Specific_Articles AS (SELECT Re.paper_id 
                                FROM Review_Notes Re WHERE 
                                Re.Focus_Location not like'%indoor %' AND 
                                Re.Focus_Location not like'%building%')
    SELECT E.edge_id, E.source_node, S.word, E.target_node, T.word, E.edge_type 
        FROM Edges E, Source_Text_Info S, Target_Text_Info T, Literature_Sources_Nodes L, Scopus_Info Sc, Specific_Articles Sp
        WHERE E.source_node = S.node_id
            AND E.target_node = T.node_id
            AND E.edge_id = L.node_id
            AND L.paper_id = Sc.paper_id
            AND Sp.paper_id = Sc.paper_id
        ORDER BY E.source_node;"""

EDGES_SOURCE_COUNT_AGGREGATED = """
    WITH Target_Nodes AS (SELECT No.node_id, No.word, COUNT(L.paper_id) AS source_count
                            FROM Literature_Sources_Nodes L, Nodes No
                            WHERE L.node_id = No.node_id
                            GROUP BY No.word
                            ORDER BY source_count DESC, No.word),
    All_Nodes AS (SELECT node_id, word FROM Nodes)
    SELECT A.word, T.word, E.edge_type, T.source_count
        FROM Edges E, Target_Nodes T, All_Nodes A
        WHERE E.target_node = T.node_id
            AND E.source_node = A.node_id
            AND E.target_node = T.node_id
        ORDER BY T.source_count DESC, T.word, A.word;
"""
EDGES_SOURCE_COUNT = """
    WITH Target_Nodes AS (SELECT No.node_id, No.word, COUNT(L.paper_id) AS source_count
                            FROM Literature_Sources_Nodes L, Nodes No
                            WHERE L.node_id = No.node_id
                            GROUP BY No.node_id
                            ORDER BY source_count DESC, No.word),
    All_Nodes AS (SELECT node_id, word FROM Nodes)
    SELECT A.word, A.node_id, T.word, T.node_id, E.edge_type, T.source_count
        FROM Edges E, Target_Nodes T, All_Nodes A
        WHERE E.target_node = T.node_id
            AND E.source_node = A.node_id
            AND E.target_node = T.node_id
        ORDER BY T.source_count DESC, T.word, A.word;
"""
SPECIFIC_ARTICLES_EDGES_SOURCE_COUNT = """
    WITH Specific_Articles AS (SELECT Re.paper_id 
                                FROM Review_Notes Re WHERE 
                                Re.Focus_Location not like '%indoor %' AND 
                                Re.Focus_Location not like '%building%'),
    Target_Nodes AS (SELECT No.node_id, No.word, COUNT(L.paper_id) AS source_count
                        FROM Literature_Sources_Nodes L, Nodes No, Specific_Articles Sp
                        WHERE L.node_id = No.node_id
                            AND L.paper_id = Sp.paper_id
                        GROUP BY No.node_id
                        ORDER BY source_count DESC, No.word),
    All_Nodes AS (SELECT node_id, word FROM Nodes)
    SELECT A.word, A.node_id, T.word, T.node_id, E.edge_type, T.source_count
        FROM Edges E, Target_Nodes T, All_Nodes A
        WHERE E.target_node = T.node_id
            AND E.source_node = A.node_id
            AND E.target_node = T.node_id
        ORDER BY T.source_count DESC, T.word, A.word;
"""
SPECIFIC_WORDS_EDGES_SOURCE_COUNT = """
    WITH Target_Nodes AS (SELECT No.node_id, No.word, COUNT(L.paper_id) AS source_count
                        FROM Literature_Sources_Nodes L, Nodes No
                        WHERE L.node_id = No.node_id
                        GROUP BY No.node_id
                        ORDER BY source_count DESC, No.word),
    All_Nodes AS (SELECT node_id, word FROM Nodes)
    SELECT A.word, A.node_id, T.word, T.node_id, E.edge_type, T.source_count
        FROM Edges E, Target_Nodes T, All_Nodes A
        WHERE E.target_node = T.node_id
            AND E.source_node = A.node_id
            AND E.target_node = T.node_id
            AND ((A.word like 'door%'
            OR A.word like '%entrance%'
            OR A.word like '%exit%'
            OR A.word like '%frontage%'
            OR A.word like '%hardware%') OR
            (T.word like 'door%'
            OR T.word like '%entrance%'
            OR T.word like '%exit%'
            OR T.word like '%frontage%'
            OR T.word like '%hardware%'))
        ORDER BY T.source_count DESC, T.word, A.word;
"""

cursor.execute(LIST_SCHEMAS)

In [11]:
result = cursor.fetchall()
for item in result:
    print(item)

('CREATE TABLE Scopus_Info(paper_id INTEGER PRIMARY KEY AUTOINCREMENT, Title TEXT, Year TEXT, Source_title TEXT, Volume TEXT, Issue TEXT, Art_No TEXT, Page_start TEXT, Page_end TEXT, Page_count TEXT, Cited_by TEXT, DOI TEXT, Link TEXT, Affiliations TEXT, Authors_with_affiliations TEXT, Abstract TEXT, Author_Keywords TEXT, Index_Keywords TEXT, Molecular_Sequence_Numbers TEXT, Chemicals_CAS TEXT, Tradenames TEXT, Manufacturers TEXT, Funding_Details TEXT, Funding_Texts TEXT, Paper_references TEXT, Correspondence_Address TEXT, Editors TEXT, Publisher TEXT, Sponsors TEXT, Conference_name TEXT, Conference_date TEXT, Conference_location TEXT, Conference_code TEXT, ISSN TEXT, ISBN TEXT, CODEN TEXT, PubMed_ID TEXT, Language_of_Original_Document TEXT, Abbreviated_Source_Title TEXT, Document_Type TEXT, Publication_Stage TEXT, Open_Access TEXT, Source TEXT, EID TEXT)',)
('CREATE TABLE Review_Notes(paper_id INTEGER REFERENCES Scopus_Info(paper_id), Primary_study_methodology_and_or_focus TEXT, Focus